In [129]:
import pandas as pd

In [130]:
final_sentence_pool = 'final_sentence_pool'
df = pd.read_csv(f"data/output/{final_sentence_pool}.csv")
print(df.shape)
df.head()

(76784, 8)


# Clean some special characters

1. '—' with '-'
2. '…' with '...'
3. remove leading whitespace

In [131]:
df['text'] = df['text'].str.replace('—', '-')
df['text'] = df['text'].str.replace('–', '-')
df['text'] = df['text'].str.replace('…', '...')
df['text'] = df['text'].str.lstrip()

In [132]:

# List of special characters
special_characters = ['—', '…', '•', '°', '©', '®', '™', '§', '†', '‡', '¶', '“', '”', '‘', '’', '«', '»', '—', '–']

# Use vectorized operations to check for special characters
contains_special_char = df['text'].apply(lambda x: any(char in x for char in special_characters))
df_with_special_chars = df[contains_special_char].copy()

# Function to find the first special character in a string
def find_special_character(text):
    for char in special_characters:
        if char in text:
            return char
    return None

# Apply the function to get the first special character
df_with_special_chars['first_special_character'] = df_with_special_chars['text'].apply(find_special_character)

# Display the 'results' DataFrame
df_with_special_chars

,text,source_party,source_name,topic,bias_estimate,model_uncertainity,sentence_id,article_id,first_special_character
6,Saks Fifth Avenue Keep your wife on-trend (esp...,Lean Right,new-york-post,Gastronomy,0,1,1be7cef8-2a16-4f2e-a6cb-4f2082cad643,945efd06-86f1-43cb-8f89-ec82154e71ed,’
8,"I spoonfed my child and, despite my being told...",Lean Left,the-guardian,Gastronomy,1,0,7257b2a2-b3ec-4c58-84dc-015c91a9e3f0,9d240858-ce10-4533-b5d8-eb9fd1cebe4b,’
10,The Department’s research also found that this...,Center,christian-science-monitor,Education,0,0,96cc2caa-db79-471e-a61d-37a6f7abd309,e12ea2c3-d1c3-42ca-80c9-36ef3a14dada,’
12,The researchers conducted a hearing loss test ...,Lean Left,nbc-news,Wellness,0,0,7593b74f-8abf-422c-9c05-04ba739411b9,545468a5-e58b-49c9-9b7b-82793cf50743,’
18,"Listen to Decoder, a show hosted by The Verge’...",Lean Left,the-verge,Automotive,0,0,8954eddc-859a-4355-b604-88a5c4edeee1,85b15198-5d51-49c7-bb62-a468d9b761a4,’
...,...,...,...,...,...,...,...,...,...
76763,"Bumstead, who routinely refurbishes MacBooks t...",Lean Left,the-verge,Conversational,0,0,718f2f1e-10d9-45f1-9894-59cfd4cf9000,5adb92a2-6da2-499c-a4ce-3a3c415fbe8a,’
76769,Biden’s chief in-party rival Robert F. Kennedy...,Right,newsbusters,Politics,0,1,0111f47a-df68-439b-97f2-ee0b5faeedc1,996a145e-adff-427d-82b8-0f54d48cf7ae,’
76777,"Meanwhile, California clings to a government-r...",Right,the-american-spectator,Politics,1,0,8ef71595-148b-43fd-9247-0825ddd7a7dd,4f7106b1-f38d-4bab-b479-9ddde32b4e1c,’
76778,The violence and vengeance in Trump’s remarks ...,Left,alternet,Politics,1,0,875b4f5e-1bbc-4d74-ae23-8a30d87c4d1a,8070258c-46fc-49d0-afaa-3a8168e69c2e,’


# Remove sentences that start with a dash or em dash

these sentences are usually corrections of previous sentences and are not useful for our purposes. Also there some links that start with a dash that we don't want to include.

This may also remove some sentences that are useful, but we can manually add those back in later. Should be a small number.

In [133]:
df['text'] = df['text'].str.replace('—', '-')
df['text'] = df['text'].str.replace('–', '-')
df['text'] = df['text'].str.replace('…', '...')
df['text'] = df['text'].str.lstrip()
df = df[~df['text'].str.startswith('-')]

In [134]:
df

,text,source_party,source_name,topic,bias_estimate,model_uncertainity,sentence_id,article_id
0,This was severely condemned by Pope Clement VI...,Lean Right,christianity-today,Disaster,1,0,dcbe6b0a-ad1a-4ad3-a267-fdce3ef9ba94,492b998c-1a9a-4bda-b746-92aeeb509068
1,Some ambitious local authorities are already f...,Lean Left,the-guardian,Conversational,1,0,d0a10b7a-8de8-430a-940b-86d6769bb551,7e294d95-e321-43fd-ba2a-cbe91af72369
2,I'm gonna go farther and even say Unicorn Barb...,Lean Left,vanity-fair,Entertainment,1,0,af87c685-dc52-4875-8493-87e59e75ad1a,8cc50f16-516e-439e-9550-33c480641b3e
3,Florida Man Doing Fine The anti-transparency a...,Lean Right,reason,Conversational,1,0,ff879773-73c1-4c49-ad46-81ac82b21237,8894eaf6-7458-4953-bdbc-bfc5964df2f4
4,The group advocates for the secession of the n...,Center,cnbc,Diplomacy,1,1,b9f9c7a7-6bcf-476f-9bdc-c5f339d7d08e,e28e1ffe-71a4-4918-b8f6-c5360595d7eb
...,...,...,...,...,...,...,...,...
76779,"Churchill died before Jonathan was born, but J...",Lean Right,the-christian-post,Conversational,0,0,d233198e-e811-4bd1-b693-26d287207cf0,017f4bfa-0ba7-4a4e-9138-af96463a848a
76780,The war spawned a wealth of fakery online: Fas...,Center,fortune,Conversational,1,0,abbda34f-f543-4958-bedf-5e62d3d4d9bd,4d77dc11-32c3-4e63-8749-259ee4308d3e
76781,But what if Fox News moderators Dana Perino an...,Left,alternet,Politics,0,1,bba983b3-5696-4d06-9b53-0de16078d324,300e033f-798c-4c5e-ab2d-48911d805921
76782,Disney is also searching for a successor to Ig...,Center,cnbc,Finance,0,0,86234e94-602a-409e-a462-70cd38327a6f,11669250-e30f-4418-9e3a-992a03b471db


# Sample 10000 sentences for MA

1. uniform distribution of sources
2. uniform distribution of bias_estimates
3. if a source has less than 10000 sentences, sample all of them

In [135]:
seed = 42


# Calculate the target number of rows per source_name
target_rows_per_source = 10000 // len(df['source_name'].unique())

sampled_data = pd.DataFrame()

for source_name, group in df.groupby('source_name'):
    # Sample half of the rows with 'bias_estimate' equal to 1
    bias_estimate_1 = group[group['bias_estimate'] == 1].sample(
        n=min(target_rows_per_source // 2, len(group[group['bias_estimate'] == 1])),
        random_state=seed
    )

    # Sample half of the rows with 'bias_estimate' equal to 0
    bias_estimate_0 = group[group['bias_estimate'] == 0].sample(
        n=min(target_rows_per_source // 2, len(group[group['bias_estimate'] == 0])),
        random_state=seed
    )
    # Concatenate the sampled data for this source_name
    sampled_data = pd.concat([sampled_data, bias_estimate_1, bias_estimate_0])

sampled_data

,text,source_party,source_name,topic,bias_estimate,model_uncertainity,sentence_id,article_id
26634,While Kellyanne Conway was a staunch defender ...,Left,alternet,Conversational,1,0,4ef2c7a4-aeb0-4472-898e-45a0ad4ae8fd,5a2356e1-1af6-4ad6-9c7a-f774324b5692
24242,"Three weeks on, the Republicans have been frit...",Left,alternet,Politics,1,0,abcb9997-033a-484e-b751-1611e43c1ce6,d6b392a2-17fa-4593-8d32-93d8723a8596
68169,"A few years ago, the far-right Britain First c...",Left,alternet,Diplomacy,1,0,9f6abed7-36e2-4eec-a5e5-bda8ce599b75,db97115d-b861-4a5b-87af-c120824d9336
18608,Jones has a history of prolific fabulism Jones...,Left,alternet,Politics,1,0,23e188e6-c2a4-4cea-907d-2e4930019450,aa072ec2-4548-403b-a5ec-ae279f086bff
54591,"The fight over masks is heated, and Goodwin, w...",Left,alternet,Politics,1,0,9cedba74-945b-4d1a-af64-fab180fe918a,27dcd49a-c9d3-4434-96ef-483a2a46c776
...,...,...,...,...,...,...,...,...
18024,While Special Counsel Jack Smith’s team and Tr...,Lean Right,zerohedge,Politics,0,0,b8d0ad8c-9bda-4ad8-ae09-d40d0e72191e,0f434173-d921-479a-ab75-d3559a501212
4652,Fiscal Restraint Amid Growing Deficits Citi's ...,Lean Right,zerohedge,Politics,0,0,614f37bf-4f82-445a-99d2-f91883517e71,435e697c-7cc4-4e4c-9237-60140fbbad74
29992,When Mr. Durbin was asked on Dec. 5 whether he...,Lean Right,zerohedge,Conversational,0,0,df68b513-9a51-4968-8186-6318c13b16eb,ab4053d0-c6e9-4fec-a64b-a4f76e3b0bb0
24175,The policy she outlined included prohibitions ...,Lean Right,zerohedge,Conversational,0,0,b1bdcdf8-8dfc-4418-829c-91fc33cf0bd1,1d176beb-c060-4ba3-891d-9bef3b5b7f57


In [136]:
sampled_data.value_counts('source_party')

source_party
Lean Left     2321
Left          2029
Lean Right    1653
Right         1536
Center        1204
dtype: int64

In [137]:
# Check if the total number of rows is less than 10,000
if len(sampled_data) < 10000:
    # Calculate the number of missing rows for each 'bias_estimate'
    missing_rows_1 = 5000 - len(sampled_data[sampled_data['bias_estimate'] == 1])
    missing_rows_0 = 5000 - len(sampled_data[sampled_data['bias_estimate'] == 0])

    # Sample the missing rows from 'Lean Right' and 'Right' source_party
    remaining_rows_1 = df[(df['source_party'].isin(['Lean Right', 'Right'])) & (df['bias_estimate'] == 1)].sample(
        n=missing_rows_1, random_state=seed
    )
    remaining_rows_0 = df[(df['source_party'].isin(['Lean Right', 'Right'])) & (df['bias_estimate'] == 0)].sample(
        n=missing_rows_0, random_state=seed
    )
    # Concatenate the remaining rows to the sampled_data
    sampled_data = pd.concat([sampled_data, remaining_rows_1, remaining_rows_0])


In [138]:
sampled_data.value_counts('source_party')

source_party
Lean Left     2321
Right         2250
Lean Right    2196
Left          2029
Center        1204
dtype: int64

In [139]:
sampled_data.value_counts('bias_estimate')

bias_estimate
0    5000
1    5000
dtype: int64

In [140]:
sampled_data.to_csv('data/output/ma_sampled_data.csv', index=False)

In [141]:
sampled_data = sampled_data.drop_duplicates()

# Assuming 'common_column' is the common column between df and sampled_data
common_column = 'text'

# Merge the dataframes using the common column
merged_df = pd.merge(df, sampled_data,  how='outer', indicator=True)

# Select rows that are in df but not in sampled_data
df_not_in_sampled_data = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)

# Now df_not_in_sampled_data contains the rows that are in df but not in sampled_data
# Print the size of each dataframe
print("Size of df:", df.shape)
print("Size of sampled_data:", sampled_data.shape)
print("Size of df_not_in_sampled_data:", df_not_in_sampled_data.shape)

# Check if the sizes sum up
total_rows_df_and_sampled_data = df_not_in_sampled_data.shape[0] + sampled_data.shape[0]

if df.shape[0] == total_rows_df_and_sampled_data:
    print("The sizes of df and sampled_data sum up correctly.")
else:
    print("The sizes of df and sampled_data do not sum up correctly.")

df_not_in_sampled_data.to_csv('data/output/not_in_ma_sample.csv', index=False)

Size of df: (76428, 8)
Size of sampled_data: (10000, 8)
Size of df_not_in_sampled_data: (66566, 8)
The sizes of df and sampled_data do not sum up correctly.


In [142]:
import numpy as np

# Specify the number of parts you want to split the DataFrame into
num_parts = 2

# Use numpy.array_split to split the DataFrame
split_dataframes = np.array_split(df_not_in_sampled_data, num_parts)

# Now, split_dataframes is a list containing the three equal parts of the original DataFrame
# Access each part like this:
part1 = split_dataframes[0].drop_duplicates()
part2 = split_dataframes[1].drop_duplicates()
#part3 = split_dataframes[2]
print("Part 1", part1.shape)
print("Part 2", part2.shape)
print(part1.shape[0] + part2.shape[0])

part1.to_csv('data/output/not_in_ma_sample_part1.csv', index=False)
part2.to_csv('data/output/not_in_ma_sample_part2.csv', index=False)

Part 1 (30711, 8)
Part 2 (30753, 8)
61464
